In [50]:
from flask import Flask, render_template, request
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
import requests
import json

In [74]:
def predict_hist(img_path):
  model_hist=load_model('/content/drive/MyDrive/Senior_Design_Project_2023/Models/Histopathology/model_1_nor.hdf5')
  img = cv2.imread(img_path ,cv2.IMREAD_COLOR)
  img_sized = cv2.resize(cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F), (50,50), interpolation = cv2.INTER_LINEAR)
  reshaped_array = np.reshape(img_sized, (1, 50, 50, 3))
  p= model_hist.predict(reshaped_array)
  if int(p.argmax())==1:
    p= "Cancer"
    return p
  else:
    p= "Normal"
    return p

In [73]:
import os
def predict_ult(img_path):
  X_nor=[]
  model_ult=load_model('/content/drive/MyDrive/Senior_Design_Project_2023/Models/Ultrasound/ult_final_model_1_aug.h5')
  img = cv2.imread(os.path.join(img_path),0)
  img=img/255
  img_size=128
  new_array = cv2.resize(img,(img_size,img_size))
  reshaped_array = np.reshape(new_array, (1, 128, 128, 1))
  p= model_ult.predict(reshaped_array)
  pred = np.argmax(p,axis=1)
  if int(pred)==0:
    p = "Normal"
    return p
  elif int(pred)==1:
    p = "Benign"
    return p
  else:
    p = "Malignant"
    return p

In [72]:
def predict_mam(img_path):
  model_mam=load_model('/content/drive/MyDrive/Senior_Design_Project_2023/Models/Mammography/model_2_mam.h5')
  image_data = tf.io.read_file(img_path)
  image = tf.io.decode_png(image_data, channels=1)  # Assuming the image is in PNG format
  (a,b,c)=image.shape
  image = np.reshape(image, (1, a, b, c))
  p= model_mam.predict(image)
  p = np.argmax(p,axis = 1)
  if int(p[0])==0:
    p = "Normal"
    return p
  else:
    p = "Cancer"
    return p

In [75]:
predict_mam('/content/image_1.png')

1/1 [==============================] - 0s 150ms/step


'Normal'

In [76]:
def save_db(patient_id,classification_type,img_path,prediction,database_url):
    patient_id = int(patient_id)
    prediction = str(prediction)
    classification_type = str(classification_type)
    img_path = str(img_path)
    image_data = {
      "patient_id": patient_id,
      "classification_type": classification_type,
      "img_path": img_path,
      "prediction": prediction
  }

    # Send a POST request to the Firebase Realtime Database
    response = requests.post(database_url + "/predictions.json", json=image_data)

In [77]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://ekswy4xu2qw-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [79]:
from flask import *
app= Flask(__name__,template_folder="/content/drive/MyDrive/Senior_Design_Project_2023/Flask/templates")

@app.route("/", methods=['GET', 'POST'])
def home():
    if request.method == "GET":
      return render_template("index.html")
    else:
      # Firebase Realtime Database URL
      database_url = 'https://breast-cancer-preds-default-rtdb.firebaseio.com'
      classification_type = request.form.get('classification_type')
      patient_id = int(request.form.get('patient_id'))
      img = request.files['my_image']
      img_path = 'static/' + img.filename
      img.save(img_path)

      if classification_type=="histopathology":
        prediction = predict_hist(img_path)
        save_db(patient_id=patient_id,classification_type=classification_type,img_path=img_path,prediction=prediction,database_url=database_url)
        return render_template('index.html',
                              prediction=prediction, img_path = img_path)
      elif classification_type=="ultrasound":
        prediction = predict_ult(img_path)
        save_db(patient_id=patient_id,classification_type=classification_type,img_path=img_path,prediction=prediction,database_url=database_url)
        return render_template('index.html',
                              prediction=prediction, img_path = img_path)
      elif classification_type=="mammography":
        prediction = predict_mam(img_path)
        save_db(patient_id=patient_id,classification_type=classification_type,img_path=img_path,prediction=prediction,database_url=database_url)
        return render_template('index.html',
                              prediction=prediction, img_path = img_path)



app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Jun/2023 07:43:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2023 07:43:43] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 140ms/step


INFO:werkzeug:127.0.0.1 - - [23/Jun/2023 07:43:52] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2023 07:43:52] "GET /static/image_1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jun/2023 07:43:53] "GET /favicon.ico HTTP/1.1" 404 -
